In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_csv('messy_data.csv')
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.rename(columns=lambda x: x.strip())
df.columns = df.columns.str.replace(' ', '_')
df['clarity'] = df['clarity'].str.upper()
df['color'] = df['color'].str.upper()
df['color'] = df['color'].replace('COLORLESS', 'D')
df['cut'] = df['cut'].str.capitalize()
df = df.replace('', np.NaN)
df['depth'] = df['depth'].astype(float)
df['table'] = df['table'].astype(float)
df['x_dimension'] = df['x_dimension'].astype(float)
df['y_dimension'] = df['y_dimension'].astype(float)
df['z_dimension'] = df['z_dimension'].astype(float)
df['price'] = df['price'].astype(float)
df['clarity'].replace(['IF', 'VVS1', 'VVS2', 'SI1', 'SI2', 'I1'],
                        [5, 4, 3, 2, 1, 0], inplace=True)
df['color'].replace(['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                        [6, 5, 4, 3, 2, 1, 0], inplace=True)
df['cut'].replace(['Fair', 'Good', 'Very good', 'Premium', 'Ideal'],
                        [0, 1, 2, 3, 4], inplace=True)
df['x_dimension'] = df['x_dimension'].fillna(round((df['z_dimension'] + df['y_dimension']) / 2, 2))
df['y_dimension'] = df['y_dimension'].fillna(round((df['x_dimension'] + df['z_dimension']) / 2, 2))
df['z_dimension'] = df['z_dimension'].fillna(round((df['x_dimension'] + df['y_dimension']) / 2, 2))
df['depth'] = df['depth'].fillna(round((df['z_dimension'] / ((df['x_dimension'] + df['y_dimension']) / 2)) * 100, 2))
df['table'] = df['table'].fillna(round((df['x_dimension'] / df['y_dimension']) * 100, 2))
df['carat'] = df['carat'].fillna(round(df['x_dimension'] * np.mean(df['carat'] / df['x_dimension']), 2))

In [3]:
fig = make_subplots(rows=2, cols=5)
fig.add_trace(go.Box(y=df['carat'], name='Carat'), row=1, col=1)
fig.add_trace(go.Box(y=df['clarity'], name='Clarity'), row=1, col=2)
fig.add_trace(go.Box(y=df['color'], name='Color'), row=1, col=3)
fig.add_trace(go.Box(y=df['cut'], name='Cut'), row=1, col=4)
fig.add_trace(go.Box(y=df['x_dimension'], name='X Dimension'), row=1, col=5)
fig.add_trace(go.Box(y=df['y_dimension'], name='Y Dimension'), row=2, col=1)
fig.add_trace(go.Box(y=df['z_dimension'], name='Z Dimension'), row=2, col=2)
fig.add_trace(go.Box(y=df['depth'], name='Depth'), row=2, col=3)
fig.add_trace(go.Box(y=df['table'], name='Table'), row=2, col=4)
fig.add_trace(go.Box(y=df['price'], name='Price'), row=2, col=5)
fig.update_layout(height=600, width=800, title_text='Variables Boxplots')
fig.show()

In [4]:
carat_Q1 = df['carat'].quantile(0.25)
carat_Q3 = df['carat'].quantile(0.75)
carat_IQR = carat_Q3 - carat_Q1
df = df[(df['carat'] >= carat_Q1 - 1.5 * carat_IQR) & (df['carat'] <= carat_Q3 + 1.5 * carat_IQR)]

price_Q1 = df['price'].quantile(0.25)
price_Q3 = df['price'].quantile(0.75)
price_IQR = price_Q3 - price_Q1
df = df[(df['price'] >= price_Q1 - 1.5 * price_IQR) & (df['price'] <= price_Q3 + 1.5 * price_IQR)]

depth_Q1 = df['depth'].quantile(0.25)
depth_Q3 = df['depth'].quantile(0.75)
depth_IQR = depth_Q3 - depth_Q1
df = df[(df['depth'] >= depth_Q1 - 1.5 * depth_IQR) & (df['depth'] <= depth_Q3 + 1.5 * depth_IQR)]

table_Q1 = df['table'].quantile(0.25)
table_Q3 = df['table'].quantile(0.75)
table_IQR = table_Q3 - table_Q1
df = df[(df['table'] >= table_Q1 - 1.5 * table_IQR) & (df['table'] <= table_Q3 + 1.5 * table_IQR)]

z_dimension_Q1 = df['z_dimension'].quantile(0.25)
z_dimension_Q3 = df['y_dimension'].quantile(0.75)
z_dimension_IQR = z_dimension_Q3 - z_dimension_Q1
df = df[(df['z_dimension'] >= z_dimension_Q1 - 1.5 * z_dimension_IQR) & (df['z_dimension'] <= z_dimension_Q3 + 1.5 * z_dimension_IQR)]

In [5]:
fig = make_subplots(rows=2, cols=5)
fig.add_trace(go.Histogram(y=df['carat'], name='Carat'), row=1, col=1)
fig.add_trace(go.Histogram(y=df['clarity'], name='Clarity'), row=1, col=2)
fig.add_trace(go.Histogram(y=df['color'], name='Color'), row=1, col=3)
fig.add_trace(go.Histogram(y=df['cut'], name='Cut'), row=1, col=4)
fig.add_trace(go.Histogram(y=df['x_dimension'], name='X Dimension'), row=1, col=5)
fig.add_trace(go.Histogram(y=df['y_dimension'], name='Y Dimension'), row=2, col=1)
fig.add_trace(go.Histogram(y=df['z_dimension'], name='Z Dimension'), row=2, col=2)
fig.add_trace(go.Histogram(y=df['depth'], name='Depth'), row=2, col=3)
fig.add_trace(go.Histogram(y=df['table'], name='Table'), row=2, col=4)
fig.add_trace(go.Histogram(y=df['price'], name='Price'), row=2, col=5)
fig.update_layout(height=600, width=1400, title_text='Variable Distributions')
fig.show()

In [6]:
fig = make_subplots(rows=2, cols=5)
fig.add_trace(go.Scatter(x=df['carat'], y=df['price'], mode='markers', name='Carat', ), row=1, col=1)
fig.add_trace(go.Scatter(x=df['clarity'], y=df['price'], mode='markers', name='Clarity'), row=1, col=2)
fig.add_trace(go.Scatter(x=df['color'], y=df['price'], mode='markers', name='Color'), row=1, col=3)
fig.add_trace(go.Scatter(x=df['cut'], y=df['price'], mode='markers', name='Cut'), row=1, col=4)
fig.add_trace(go.Scatter(x=df['x_dimension'], y=df['price'], mode='markers', name='X Dimension'), row=1, col=5)
fig.add_trace(go.Scatter(x=df['y_dimension'], y=df['price'], mode='markers', name='Y Dimension'), row=2, col=1)
fig.add_trace(go.Scatter(x=df['z_dimension'], y=df['price'], mode='markers', name='Z Dimension'), row=2, col=2)
fig.add_trace(go.Scatter(x=df['depth'], y=df['price'], mode='markers', name='Depth'), row=2, col=3)
fig.add_trace(go.Scatter(x=df['table'], y=df['price'], mode='markers', name='Table'), row=2, col=4)
fig.update_layout(height=600, width=1400, title_text='Price vs. Features')
fig.show()

In [7]:
fig = px.scatter(df, x='carat', y='price', color='clarity', marginal_y='violin')
fig.update_layout(title='Price vs. Carat by Clarity')
fig.show()

In [8]:
fig = px.imshow(df.corr(), color_continuous_scale='RdBu')
fig.update_layout(title='Correlation Matrix')
fig.show()

In [9]:
#z uwagi na silną współliniowość wymiarów x, y, z i carat, uwzględniam tylko carat (jest on zależny od każdego z wymiarów)
attributes = ['carat', 'table', 'clarity', 'depth', 'cut', 'color']
formula = 'price ~'
for attribute in attributes:
    if formula[-1] == '~':
        formula += ' ' + attribute
    else:
        formula += ' + ' + attribute
    lm = smf.ols(formula=formula, data=df)
    lm_fit = lm.fit()
    print('Formula: ' + formula)
    print('R-squared: ' + str(round(lm_fit.rsquared, 4)))

Formula: price ~ carat
R-squared: 0.8453
Formula: price ~ carat + table
R-squared: 0.8539
Formula: price ~ carat + table + clarity
R-squared: 0.905
Formula: price ~ carat + table + clarity + depth
R-squared: 0.9051
Formula: price ~ carat + table + clarity + depth + cut
R-squared: 0.9061
Formula: price ~ carat + table + clarity + depth + cut + color
R-squared: 0.9063


In [10]:
model = smf.ols(formula='price ~ carat + table + clarity + depth + cut + color', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     216.0
Date:                Sun, 04 Feb 2024   Prob (F-statistic):           2.51e-66
Time:                        01:05:32   Log-Likelihood:                -1165.8
No. Observations:                 141   AIC:                             2346.
Df Residuals:                     134   BIC:                             2366.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3994.5474   1.71e+04     -0.234      0.8

In [11]:
X = df[['carat', 'color', 'cut', 'clarity', 'depth', 'table']]
vif_data = pd.DataFrame() 
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))]
print(vif_data)

   feature          VIF
0    carat     9.711159
1    color     5.084745
2      cut     3.273010
3  clarity     3.280227
4    depth  1798.446377
5    table  1918.966646


In [12]:
#wysoki współczynnik VIF oznacza współliniowość między predyktorami, więc usuwam początkowo depth i table
model = smf.ols(formula='price ~ carat + clarity + cut + color', data=df).fit()
print(model.summary())

X = df[['carat', 'color', 'cut', 'clarity']]
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     323.6
Date:                Sun, 04 Feb 2024   Prob (F-statistic):           2.01e-68
Time:                        01:05:32   Log-Likelihood:                -1166.8
No. Observations:                 141   AIC:                             2344.
Df Residuals:                     136   BIC:                             2358.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3876.6540    304.191    -12.744      0.0

In [13]:
X = df[['carat', 'color', 'cut', 'clarity']]
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = smf.ols(formula='price ~ carat + clarity + cut + color', data=df).fit()
y_pred = model.predict(X_test)

fig = px.scatter(x=y_test, y=y_pred)
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Regression line'))
fig.update_layout(title='Predicted vs Actual Price', xaxis_title='Actual', yaxis_title='Predicted', height=600, width=800)
fig.show()  